In [1]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot, make_vio_plot_label

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
#define widgets
#modality_widget=widgets.Dropdown(
#    options=['bold', 'structural'],
#    description='Modality:',
#    disabled=False)


modality_widget=widgets.RadioButtons(
    options=['bold', 'structural'],
    description='Modality:',
    disabled=False
)

TR_min=widgets.FloatSlider(
    min=1.5,
    max=5,
    step=0.1,
    description='TR min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


TR_max=widgets.FloatSlider(
    min=1.5,
    max=4,
    step=0.1,
    description='TR max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)



TE_min=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


TE_max=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


select_parameters=widgets.SelectMultiple(
    options=['TR_min', 'TR_max', 'TE_min', 'TE_max'],
    description='Parameters',
    disabled=False)

#we are at 2.4, 0.0041

In [3]:
display(modality_widget)

RadioButtons(description='Modality:', options=('bold', 'structural'), value='bold')

In [5]:
modal=modality_widget.value
display(select_parameters)
"If empty, no parameter restrictions will applied"

SelectMultiple(description='Parameters', index=(0, 1), options=('TR_min', 'TR_max', 'TE_min', 'TE_max'), value…

'If empty, no parameter restrictions will applied'

In [7]:
for i in select_parameters.value:
        display(eval(i))

FloatSlider(value=2.2, continuous_update=False, description='TR min:', max=5.0, min=1.5, readout_format='')

FloatSlider(value=2.6, continuous_update=False, description='TR max:', max=4.0, min=1.5, readout_format='')

In [8]:
to_filter=select_parameters.value
filter_list=[]
filter_dict={'TR_min': "TR >= {}".format(TR_min.value),
             'TR_max': "TR < {}".format(TR_max.value),
             'TE_min': "TE >= {}".format(TE_min.value),
             'TE_max': "TE < {}".format(TE_max.value)
            }

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)
    
print(filter_list)
#filter_list= ['TR > 2.2', 'TR < 3']

['TR >= 2.2', 'TR < 2.6']


In [9]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd
homedir = os.path.expanduser('~/OBIWAN')
# path to input of local data from MRIQC on your own dataset 
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'group_bold.tsv')
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC','afterPREPROC',  'group_bold.tsv')
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'group_T1w.tsv')

#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'obese_bold.tsv')
group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'control_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'
#modality = 'T1w'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   Tesla, TE, TR
   NOTE: Only working as *and* right now!
"""


# IQM variables to visualize
#need to add separate IQMs for structural and functional
IQM_to_plot = ['aor']#,'aqi','dvars_nstd', 'tsnr', 'snr'] #'cjv','cnr', 'efc', 'snr_gm', 'snr_csf', 'rpve_gm', 'fwhm_avg'] # 'aor','aqi','dvars_nstd', 'tsnr', 'snr'
                    #'dvars_std','dvars_vstd',
                    #'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
                    #'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
                    #'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
                    #'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
                    #'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
                    #'summary_fg_p95','summary_fg_stdv','dummy_trs']
                    
                      # variable names we might want to list
    #qc_var_list = ["aor","aqi","cjv","cnr","dummy_trs","dvars_nstd","dvars_std","dvars_vstd","efc","fber","fber","fd_mean","fd_num","fd_perc","fwhm_avg","fwhm_avg","fwhm_x","fwhm_y","fwhm_z","gcor","gsr_x","gsr_y","icvs_csf","icvs_gm",
#"icvs_wm","inu_med","inu_range","qi_1","qi_2","rpve_csf","rpve_gm","rpve_wm","snr", "snr_csf","snr_gm","snr_total","snr_wm","snrd_csf","snrd_gm","snrd_total","snrd_wm","summary_bg_k","summary_bg_mad","summary_bg_mean","summary_bg_median","summary_bg_n",
#"summary_bg_p05","summary_bg_p95","summary_bg_stdv","summary_csf_k","summary_csf_mad","summary_csf_mean","summary_csf_median","summary_csf_n","summary_csf_p05","summary_csf_p95","summary_csf_stdv","summary_fg_k","summary_fg_mad","summary_fg_mean",
#"summary_fg_median","summary_fg_n","summary_fg_p05","summary_fg_p95","summary_fg_stdv","summary_gm_k","summary_gm_mad","summary_gm_mean","summary_gm_median","summary_gm_n","summary_gm_p05","summary_gm_p95","summary_gm_stdv","summary_wm_k","summary_wm_mad",
#"summary_wm_mean","summary_wm_median","summary_wm_n","summary_wm_p05","summary_wm_p95","summary_wm_stdv","tpm_overlap_csf","tpm_overlap_gm","tpm_overlap_wm","tsnr","wm2max"]

In [16]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape


(127, 45)

In [22]:
userdf = userdf.loc[0:10,:]

In [23]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
if not filter_list == []:
    filtered_apidf = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(list(filtered_apidf))

(1000, 77)
(51, 77)


In [24]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



(62, 78)

In [25]:
# Visualization # 

# Create a violin plot of the MRIQC metrics, comparing user level-data to API data 
# Also will print out a short description of the measure and what the appropriate range/values should be 
# Input the merged dataframe including the API and user data
# Optional arguments include a list of specific variables that the user wants displayed 
# If no optional arguments included, will plot all measures 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"", 'Control', outliers=False)
#v = make_vio_plot_label(vis_ready_df,IQM_to_plot,"", 'Control', 'mriQC API', outliers=False)
#after_preproc = v[2]
#widgets.VBox([v[0],v[1]])


Loading in dataframe...
Loading variables: ['aor']
Loading in data descriptors...


In [26]:

def make_vio_plot_label(data, IQM_to_plot, data_descriptors, label1, label2, outliers=False):
    ''' Make a violin plot of the api and user QC metrics.

    Args:
        data (dataframe): a dataframe including the API and USER data. Must have a column labeled 'source' with USER or API defined.
        IQM_to_plot (list): list of IQMs to plot. If you want to view all the IQMs, leave the list empty.
        data_descriptors (path-to-csv): the path to read in a csv of variable descriptions
        outliers (Boolean): if True, remove outliers. Default is to leave outliers in

    Returns: A violin plot of each MRIQC metric, comparing the user-level data to
    the API data.

    '''
    print('Loading in dataframe...')

    # variable names we might want to list
    qc_var_list = ["aor",
"aqi",
"cjv",
"cnr",
"dummy_trs",
"dvars_nstd",
"dvars_std",
"dvars_vstd",
"efc",
"fber",
"fber",
"fd_mean",
"fd_num",
"fd_perc",
"fwhm_avg",
"fwhm_avg",
"fwhm_x",
"fwhm_y",
"fwhm_z",
"gcor",
"gsr_x",
"gsr_y",
"icvs_csf",
"icvs_gm",
"icvs_wm",
"inu_med",
"inu_range",
"qi_1",
"qi_2",
"rpve_csf",
"rpve_gm",
"rpve_wm",
"snr",
"snr_csf",
"snr_gm",
"snr_total",
"snr_wm",
"snrd_csf",
"snrd_gm",
"snrd_total",
"snrd_wm",
"summary_bg_k",
"summary_bg_mad",
"summary_bg_mean",
"summary_bg_median",
"summary_bg_n",
"summary_bg_p05",
"summary_bg_p95",
"summary_bg_stdv",
"summary_csf_k",
"summary_csf_mad",
"summary_csf_mean",
"summary_csf_median",
"summary_csf_n",
"summary_csf_p05",
"summary_csf_p95",
"summary_csf_stdv",
"summary_fg_k",
"summary_fg_mad",
"summary_fg_mean",
"summary_fg_median",
"summary_fg_n",
"summary_fg_p05",
"summary_fg_p95",
"summary_fg_stdv",
"summary_gm_k",
"summary_gm_mad",
"summary_gm_mean",
"summary_gm_median",
"summary_gm_n",
"summary_gm_p05",
"summary_gm_p95",
"summary_gm_stdv",
"summary_wm_k",
"summary_wm_mad",
"summary_wm_mean",
"summary_wm_median",
"summary_wm_n",
"summary_wm_p05",
"summary_wm_p95",
"summary_wm_stdv",
"tpm_overlap_csf",
"tpm_overlap_gm",
"tpm_overlap_wm",
"tsnr",
"wm2max"]

    # add stuff about whether or not variables were defined

    if len(IQM_to_plot) == 0:
        variables = qc_var_list
        print('Loading all variables...')
    elif len(IQM_to_plot) > 0:
        for x in IQM_to_plot:
            if str(x) not in qc_var_list:
                print('Variable name not recognized.')
                sys.exit()
            else:
                pass
        variables = IQM_to_plot
        print('Loading variables: %s' %variables)

    # data descriptor stuff
    print('Loading in data descriptors...')

    descriptors = pd.read_csv('./tools/iqm_descriptions.csv')

    #if not outliers:
    #    print('Please specify whether you want api outliers in your visualization or not')

    # source: user/api
    # change the file from short format to long format
    df_long = pd.melt(data, id_vars=['bids_name','SOURCE'],var_name='var',value_name='values')


    # make plotting dictionary for family colors
    # mediumpurple - lightskyblue - red is #A52A2A - orange is #D2691E - yellow is #DAA520 - lightseafoamgreen -
    plot_dict = {'tsnr': ('#D2691E'), 'gcor': ('#D2691E'), 'dvars_vstd': ('#D2691E'), 'dvars_std': ('#D2691E'), # temporal
                 'dvars_nstd': ('#D2691E'),
                 'fwhm_x': ('#DAA520'), 'fwhm_y': ('#DAA520'), 'fwhm_z': ('#DAA520'), 'fwhm_avg': ('#DAA520'), #spatial
                 'fber': ('#DAA520'), 'efc': ('#DAA520'),
                 'cjv': ('#A52A2A'), 'cnr': ('#A52A2A'), 'qi_2': ('#A52A2A'), 'snr': ('#A52A2A'), # noise
                 'snr_csf': ('#A52A2A'), 'snr_gm': ('#A52A2A'), 'snr_wm': ('#A52A2A'), 'snr_total': ('#A52A2A'),
                 'snrd_csf': ('#A52A2A'), 'snrd_gm': ('#A52A2A'), 'snrd_wm': ('#A52A2A'),
                 'fd_mean': ('#66CDAA'), 'fd_num': ('#66CDAA'), 'fd_perc': ('#66CDAA'), # motion IQMs
                 'inu_med': ('#6495ED'), 'inu_range': ('#6495ED'), 'wm2max': ('#6495ED'), # artifact IQMs
                 'aor': ('#9932CC'), 'aqi': ('#9932CC'), 'dummy_trs': ('#9932CC'), 'gsr_x': ('#9932CC'), # other
                 'gsr_y': ('#9932CC'), 'qi_1': ('#9932CC'), 'rpve_csf': ('#9932CC'), 'rpve_gm': ('#9932CC'),
                 'rpve_wm': ('#9932CC'), 'tpm_overlap_csf': ('#9932CC'), 'tpm_overlap_gm': ('#9932CC'),
                 'tpm_overlap_wm': ('#9932CC'),
                 'icvs_csf': ('#00008B'), 'icvs_gm': ('#00008B'), 'icvs_wm': ('#00008B'), # descriptive
                 'summary_bg_k': ('#00008B'), 'summary_bg_mad': ('#00008B'), 'summary_bg_mean': ('#00008B'),
                 'summary_bg_median': ('#00008B'), 'summary_bg_n': ('#00008B'), 'summary_bg_p05': ('#00008B'),
                 'summary_bg_p95': ('#00008B'), 'summary_bg_stdv': ('#00008B'),
                 'summary_csf_k': ('#00008B'), 'summary_csf_mad': ('#00008B'), 'summary_csf_mean': ('#00008B'),
                 'summary_csf_median': ('#00008B'), 'summary_csf_n': ('#00008B'), 'summary_csf_p05': ('#00008B'),
                 'summary_csf_p95': ('#00008B'), 'summary_csf_stdv': ('#00008B'),
                 'summary_fg_k': ('#00008B'), 'summary_fg_mad': ('#00008B'), 'summary_fg_mean': ('#00008B'),
                 'summary_fg_median': ('#00008B'), 'summary_fg_n': ('#00008B'), 'summary_fg_p05': ('#00008B'),
                 'summary_fg_p95': ('#00008B'), 'summary_fg_stdv': ('#00008B'),
                 'summary_gm_k': ('#00008B'), 'summary_gm_mad': ('#00008B'), 'summary_gm_mean': ('#00008B'),
                 'summary_gm_median': ('#00008B'), 'summary_gm_n': ('#00008B'), 'summary_gm_p05': ('#00008B'),
                 'summary_gm_p95': ('#00008B'), 'summary_gm_stdv': ('#00008B'),
                 'summary_wm_k': ('#00008B'), 'summary_wm_mad': ('#00008B'), 'summary_wm_mean': ('#00008B'),
                 'summary_wm_median': ('#00008B'), 'summary_wm_n': ('#00008B'), 'summary_wm_p05': ('#00008B'),
                 'summary_wm_p95': ('#00008B'), 'summary_wm_stdv': ('#00008B')
                 }

    var_name = variables[0] # the default first variable
    API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']


    def remove_outliers_from_api(API_data,outliers):

        # identify some outliers
        if outliers:
            q75, q25 = np.percentile(API_data, [75 ,25])
            iqr = q75 - q25
            min_out = q25-1.5*iqr
            max_out = q75+1.5*iqr

            #if outliers are present, replace them with NaN
            API_data[API_data > max_out] = np.nan
            API_data[API_data < min_out] = np.nan

        return API_data

    API_data = remove_outliers_from_api(API_data,outliers)

    def make_range(data, variable):
        mini_data = data.loc[(data['var']==variable)]
        max_point = mini_data.max().values[3]
        min_point = mini_data.min().values[3]
        spread = max_point-min_point
        point_range = [(min_point-(.2*spread)), (max_point+(.2*spread))]
        return point_range
                    
    # create a split violin plot for a single variable
    fig = go.Figure()

    fig.add_trace(go.Violin(x=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'var'],
                    y=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values'],
                    legendgroup=label1, scalegroup=label1, name=label1,
                    side='negative',
                    points='all',
                    pointpos=-0.5, # where to position points
                    jitter=0.1,
                    hovertext=df_long['bids_name'],
                    line_color=plot_dict.get(var_name, 'red')) # plot same-family IQMs in same color)
         )
    fig.add_trace(go.Violin(x=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'var'],
                    y=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values'],
                    legendgroup=label2, scalegroup=label2, name=label2,
                    side='positive',
                    points='all',
                    pointpos=0.5, # where to position points
                    jitter=0.1,
                    hovertext=df_long['bids_name'],
                    line_color='rgb(58,54,54)')
         )
    # update characteristics shared by all traces
    definition = descriptors.loc[(descriptors['iqm_name']==var_name),"iqm_definition"].tolist()[0]
    fig.update_traces(meanline_visible=True,
              box_visible=True)
    fig.update_layout(autosize=False,
                     width=700,
                     height=700,
                     margin=go.layout.Margin(t=0),
                     xaxis=go.layout.XAxis(title = go.layout.xaxis.Title(text=definition,font=dict(size=12))))
    fig.update_yaxes(range=make_range(df_long, var_name))
    fig.update_layout(template="plotly_white") # make background white

    # create a figure widget in order to show the dropdown menu
    fig_widget = go.FigureWidget(fig)

    # create a dropdown menu widget for the variable name
    dropdown_widget = widgets.Dropdown(
            options=variables,
            value=var_name,
            description='IQM:',
            )
    # define the changes that occur based on the choice in the dropdown menu
    def response(change):
        var_name = dropdown_widget.value

        API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']
        API_data = remove_outliers_from_api(API_data,outliers)
        user_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values']

        with fig_widget.batch_update():
            fig_widget.data[0].x = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'var']
            fig_widget.data[0].y = user_data
            fig_widget.data[0].line = {'color': plot_dict.get(var_name, 'red')}
            fig_widget.data[1].x = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'var']
            fig_widget.data[1].y = API_data
            definition = descriptors.loc[(descriptors['iqm_name']==var_name),"iqm_definition"].tolist()[0]
            fig_widget.layout.xaxis.title = {'text': definition, 'font': {'size':12}}
            fig_widget.layout.yaxis.range = make_range(df_long, var_name)
    
    def df():
        var_name = dropdown_widget.value

        API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']
        API_data = remove_outliers_from_api(API_data,outliers)
        user_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values']
        return user_data
    
    data = df()
    dropdown_widget.observe(response, names="value")

    return(dropdown_widget, fig_widget, data)



In [27]:
def make_vio_plot(data, IQM_to_plot, data_descriptors, label1, outliers=False):
    ''' Make a violin plot of the api and user QC metrics.

    Args:
        data (dataframe): a dataframe including the API and USER data. Must have a column labeled 'source' with USER or API defined.
        IQM_to_plot (list): list of IQMs to plot. If you want to view all the IQMs, leave the list empty.
        data_descriptors (path-to-csv): the path to read in a csv of variable descriptions
        outliers (Boolean): if True, remove outliers. Default is to leave outliers in

    Returns: A violin plot of each MRIQC metric, comparing the user-level data to
    the API data.

    '''
    print('Loading in dataframe...')

    # variable names we might want to list
    qc_var_list = ["aor",
"aqi",
"cjv",
"cnr",
"dummy_trs",
"dvars_nstd",
"dvars_std",
"dvars_vstd",
"efc",
"fber",
"fber",
"fd_mean",
"fd_num",
"fd_perc",
"fwhm_avg",
"fwhm_avg",
"fwhm_x",
"fwhm_y",
"fwhm_z",
"gcor",
"gsr_x",
"gsr_y",
"icvs_csf",
"icvs_gm",
"icvs_wm",
"inu_med",
"inu_range",
"qi_1",
"qi_2",
"rpve_csf",
"rpve_gm",
"rpve_wm",
"snr",
"snr_csf",
"snr_gm",
"snr_total",
"snr_wm",
"snrd_csf",
"snrd_gm",
"snrd_total",
"snrd_wm",
"summary_bg_k",
"summary_bg_mad",
"summary_bg_mean",
"summary_bg_median",
"summary_bg_n",
"summary_bg_p05",
"summary_bg_p95",
"summary_bg_stdv",
"summary_csf_k",
"summary_csf_mad",
"summary_csf_mean",
"summary_csf_median",
"summary_csf_n",
"summary_csf_p05",
"summary_csf_p95",
"summary_csf_stdv",
"summary_fg_k",
"summary_fg_mad",
"summary_fg_mean",
"summary_fg_median",
"summary_fg_n",
"summary_fg_p05",
"summary_fg_p95",
"summary_fg_stdv",
"summary_gm_k",
"summary_gm_mad",
"summary_gm_mean",
"summary_gm_median",
"summary_gm_n",
"summary_gm_p05",
"summary_gm_p95",
"summary_gm_stdv",
"summary_wm_k",
"summary_wm_mad",
"summary_wm_mean",
"summary_wm_median",
"summary_wm_n",
"summary_wm_p05",
"summary_wm_p95",
"summary_wm_stdv",
"tpm_overlap_csf",
"tpm_overlap_gm",
"tpm_overlap_wm",
"tsnr",
"wm2max"]

    # add stuff about whether or not variables were defined

    if len(IQM_to_plot) == 0:
        variables = qc_var_list
        print('Loading all variables...')
    elif len(IQM_to_plot) > 0:
        for x in IQM_to_plot:
            if str(x) not in qc_var_list:
                print('Variable name not recognized.')
                sys.exit()
            else:
                pass
        variables = IQM_to_plot
        print('Loading variables: %s' %variables)

    # data descriptor stuff
    print('Loading in data descriptors...')

    descriptors = pd.read_csv('./tools/iqm_descriptions.csv')

    #if not outliers:
    #    print('Please specify whether you want api outliers in your visualization or not')

    # source: user/api
    # change the file from short format to long format
    df_long = pd.melt(data, id_vars=['bids_name','SOURCE'],var_name='var',value_name='values')


    # make plotting dictionary for family colors
    # mediumpurple - lightskyblue - red is #A52A2A - orange is #D2691E - yellow is #DAA520 - lightseafoamgreen -
    plot_dict = {'tsnr': ('#D2691E'), 'gcor': ('#D2691E'), 'dvars_vstd': ('#D2691E'), 'dvars_std': ('#D2691E'), # temporal
                 'dvars_nstd': ('#D2691E'),
                 'fwhm_x': ('#DAA520'), 'fwhm_y': ('#DAA520'), 'fwhm_z': ('#DAA520'), 'fwhm_avg': ('#DAA520'), #spatial
                 'fber': ('#DAA520'), 'efc': ('#DAA520'),
                 'cjv': ('#A52A2A'), 'cnr': ('#A52A2A'), 'qi_2': ('#A52A2A'), 'snr': ('#A52A2A'), # noise
                 'snr_csf': ('#A52A2A'), 'snr_gm': ('#A52A2A'), 'snr_wm': ('#A52A2A'), 'snr_total': ('#A52A2A'),
                 'snrd_csf': ('#A52A2A'), 'snrd_gm': ('#A52A2A'), 'snrd_wm': ('#A52A2A'),
                 'fd_mean': ('#66CDAA'), 'fd_num': ('#66CDAA'), 'fd_perc': ('#66CDAA'), # motion IQMs
                 'inu_med': ('#6495ED'), 'inu_range': ('#6495ED'), 'wm2max': ('#6495ED'), # artifact IQMs
                 'aor': ('#9932CC'), 'aqi': ('#9932CC'), 'dummy_trs': ('#9932CC'), 'gsr_x': ('#9932CC'), # other
                 'gsr_y': ('#9932CC'), 'qi_1': ('#9932CC'), 'rpve_csf': ('#9932CC'), 'rpve_gm': ('#9932CC'),
                 'rpve_wm': ('#9932CC'), 'tpm_overlap_csf': ('#9932CC'), 'tpm_overlap_gm': ('#9932CC'),
                 'tpm_overlap_wm': ('#9932CC'),
                 'icvs_csf': ('#00008B'), 'icvs_gm': ('#00008B'), 'icvs_wm': ('#00008B'), # descriptive
                 'summary_bg_k': ('#00008B'), 'summary_bg_mad': ('#00008B'), 'summary_bg_mean': ('#00008B'),
                 'summary_bg_median': ('#00008B'), 'summary_bg_n': ('#00008B'), 'summary_bg_p05': ('#00008B'),
                 'summary_bg_p95': ('#00008B'), 'summary_bg_stdv': ('#00008B'),
                 'summary_csf_k': ('#00008B'), 'summary_csf_mad': ('#00008B'), 'summary_csf_mean': ('#00008B'),
                 'summary_csf_median': ('#00008B'), 'summary_csf_n': ('#00008B'), 'summary_csf_p05': ('#00008B'),
                 'summary_csf_p95': ('#00008B'), 'summary_csf_stdv': ('#00008B'),
                 'summary_fg_k': ('#00008B'), 'summary_fg_mad': ('#00008B'), 'summary_fg_mean': ('#00008B'),
                 'summary_fg_median': ('#00008B'), 'summary_fg_n': ('#00008B'), 'summary_fg_p05': ('#00008B'),
                 'summary_fg_p95': ('#00008B'), 'summary_fg_stdv': ('#00008B'),
                 'summary_gm_k': ('#00008B'), 'summary_gm_mad': ('#00008B'), 'summary_gm_mean': ('#00008B'),
                 'summary_gm_median': ('#00008B'), 'summary_gm_n': ('#00008B'), 'summary_gm_p05': ('#00008B'),
                 'summary_gm_p95': ('#00008B'), 'summary_gm_stdv': ('#00008B'),
                 'summary_wm_k': ('#00008B'), 'summary_wm_mad': ('#00008B'), 'summary_wm_mean': ('#00008B'),
                 'summary_wm_median': ('#00008B'), 'summary_wm_n': ('#00008B'), 'summary_wm_p05': ('#00008B'),
                 'summary_wm_p95': ('#00008B'), 'summary_wm_stdv': ('#00008B')
                 }

    var_name = variables[0] # the default first variable
    API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']


    def remove_outliers_from_api(API_data,outliers):

        # identify some outliers
        if outliers:
            q75, q25 = np.percentile(API_data, [75 ,25])
            iqr = q75 - q25
            min_out = q25-1.5*iqr
            max_out = q75+1.5*iqr

            #if outliers are present, replace them with NaN
            API_data[API_data > max_out] = np.nan
            API_data[API_data < min_out] = np.nan

        return API_data

    API_data = remove_outliers_from_api(API_data,outliers)

    def make_range(data, variable):
        mini_data = data.loc[(data['var']==variable)]
        max_point = mini_data.max().values[3]
        min_point = mini_data.min().values[3]
        spread = max_point-min_point
        point_range = [(min_point-(.2*spread)), (max_point+(.2*spread))]
        return point_range
                    
    # create a split violin plot for a single variable
    print('Ploting...')
    fig = go.Figure()

    fig.add_trace(go.Violin(x=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'var'],
                    y=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values'],
                    legendgroup=label1, scalegroup=label1, name=label1,
                    side='negative',
                    points='all',
                    pointpos=-0.5, # where to position points
                    jitter=0.1,
                    hovertext=df_long['bids_name'],
                    line_color=plot_dict.get(var_name, 'red')) # plot same-family IQMs in same color)
         )
    fig.add_trace(go.Violin(x=df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'var'],
                    y=API_data,
                    legendgroup='mriQC API', scalegroup='mriQC API', name='mriQC API',
                    side='positive',
                    line_color='rgb(58,54,54)')
         )
    # update characteristics shared by all traces
    definition = descriptors.loc[(descriptors['iqm_name']==var_name),"iqm_definition"].tolist()[0]
    fig.update_traces(meanline_visible=True,
              box_visible=True)
    fig.update_layout(autosize=False,
                     width=700,
                     height=700,
                     margin=go.layout.Margin(t=0),
                     xaxis=go.layout.XAxis(title = go.layout.xaxis.Title(text=definition,font=dict(size=12))))
    fig.update_yaxes(range=make_range(df_long, var_name))
    fig.update_layout(template="plotly_white") # make background white

    # create a figure widget in order to show the dropdown menu
    fig_widget = go.FigureWidget(fig)

    # create a dropdown menu widget for the variable name
    dropdown_widget = widgets.Dropdown(
            options=variables,
            value=var_name,
            description='IQM:',
            )
    # define the changes that occur based on the choice in the dropdown menu
    def response(change):
        var_name = dropdown_widget.value

        API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']
        API_data = remove_outliers_from_api(API_data,outliers)
        user_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values']

        with fig_widget.batch_update():
            fig_widget.data[0].x = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'var']
            fig_widget.data[0].y = user_data
            fig_widget.data[0].line = {'color': plot_dict.get(var_name, 'red')}
            fig_widget.data[1].x = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'var']
            fig_widget.data[1].y = API_data
            definition = descriptors.loc[(descriptors['iqm_name']==var_name),"iqm_definition"].tolist()[0]
            fig_widget.layout.xaxis.title = {'text': definition, 'font': {'size':12}}
            fig_widget.layout.yaxis.range = make_range(df_long, var_name)
    
    def df():
        var_name = dropdown_widget.value

        API_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='API'),'values']
        API_data = remove_outliers_from_api(API_data,outliers)
        user_data = df_long.loc[(df_long['var']==var_name)&(df_long['SOURCE']=='USER'),'values']
        return user_data
    
    data = df()
    dropdown_widget.observe(response, names="value")

    return(dropdown_widget, fig_widget, data)


    #print description of figure

    #print(dictionary.get(var_name))